In [118]:
import pandas as pd
import requests

In [119]:
year = '2023-01-01'
params = {
    'page[size]': '100',
    'page[number]': '1',
    'filter': f'record_date:gte:{year}'
}
endpoint = '/v1/accounting/dts/deposits_withdrawals_operating_cash'
base_url = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service'
url = base_url + endpoint
all_data = []

In [120]:
class Data:
    def __init__(self, url: str, page_size=None, params=None) -> None:
        self.all_data = []
        self.df = None
        self.url = url
        self.params = params
        self.page_size = page_size
        self.columns = {}
        self.last_accessed = None
    
    def search(self) -> pd.DataFrame:
        previous_response_status_code = None
        previous_page_size = self.page_size
        
        while True:
            response = requests.get(self.url, params=self.params)
            
            # if previous_response_status_code == None:
            #     previous_response_status_code = response.status_code
            
            if response.status_code != 200 and previous_response_status_code != None:
                print(response.status_code)
                if previous_response_status_code != 200:
                    print("All data gathered")
                    break
                else:
                    self.page_size = int(params['page[size]'])
                    print(f'Failed, {self.page_size}')
                    params['page[size]'] = str((self.page_size + previous_page_size) // 2)
                    continue

            
            page_data = response.json().get('data', [])
            
            if not page_data:
                break
            
            self.all_data.extend(page_data)
            
            # Sets the column names
            
            previous_response_status_code = response.status_code
            print(f'Successful, {self.page_size}')
            print(f'Param before: {params["page[size]"]}')
            params['page[size]'] = str(self.page_size * 2)
            print(f'Param after: {params["page[size]"]}')
            params['page[number]'] = str(int(params['page[number]']) + 1)
        params['page[number]'] = '1'
        
        if self.all_data:
            df = pd.DataFrame(self.all_data)
            self.df = df
            for column in self.df.columns:
                self.columns[column.replace('_', ' ').capitalize()] = column
            print(df.shape)
        else:
            print('No data found')

        print(params['page[size]'])
        return
    
    def list_columns(self):
        for column in self.columns:
            print(column)
    
    def view_data(self):
        print(self.df)

In [121]:
data = Data(url,100, params)
data.search()

Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100
Successful, 100


KeyboardInterrupt: 

In [105]:
data.df.describe()

,record_date,account_type,transaction_type,transaction_catg,transaction_catg_desc,transaction_today_amt,transaction_mtd_amt,transaction_fytd_amt,table_nbr,table_nm,src_line_nbr,record_fiscal_year,record_fiscal_quarter,record_calendar_year,record_calendar_quarter,record_calendar_month,record_calendar_day
count,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700,12700
unique,78,3,2,144,7,1515,3906,7269,1,1,184,2,3,2,3,7,31
top,2023-03-01,Treasury General Account (TGA),Withdrawals,Int'l Assistance Programs (IAP) - misc,null,0,0,1,II,Deposits and Withdrawals of Operating Cash,33,2024,1,2023,4,12,27
freq,182,12558,7137,144,12652,2536,892,113,12700,12700,73,6400,6361,12661,6361,3620,715


In [106]:
# df['transaction_today_amt'] = pd.to_numeric(df['transaction_today_amt'])
# df_sorted = df['transaction_today_amt'].sort_values(ascending=False)
# print(df_sorted)